# Week 1 Homework

In [1]:
import pandas as pd

## Q1. Downloading the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [2]:
!wget "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet" -O "./data/yellow_tripdata_2023-01.parquet"
!wget "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet" -O "./data/yellow_tripdata_2023-02.parquet"

--2025-05-19 11:51:53--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 65.8.245.171, 65.8.245.178, 65.8.245.51, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|65.8.245.171|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47673370 (45M) [application/x-www-form-urlencoded]
Saving to: ‘./data/yellow_tripdata_2023-01.parquet’

./data/yellow_tripd 100%[===================>]  45.46M   186MB/s    in 0.2s    

2025-05-19 11:51:53 (186 MB/s) - ‘./data/yellow_tripdata_2023-01.parquet’ saved [47673370/47673370]

--2025-05-19 11:51:53--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 65.8.245.171, 65.8.245.178, 65.8.245.51, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|65.8.245.171|:443... conn

In [3]:
df_jan = pd.read_parquet("./data/yellow_tripdata_2023-01.parquet")

In [4]:
df_jan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [5]:
# Or
len(df_jan.columns)

19

### Answer:
There are 19 columns in the January dataframe!

***

## Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [6]:
df_jan['duration'] = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime
df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds() / 60.0)

In [7]:
df_jan.duration

,duration
0,8.433333
1,6.316667
2,12.750000
3,9.616667
4,10.833333
...,...
3066761,13.983333
3066762,19.450000
3066763,24.516667
3066764,13.000000


In [8]:
df_jan.duration.std()

42.594351241920904

### Answer:
Standard deviation of trips in January is 42.59

***

## Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [9]:
# Length before dropping outliers
total_jan_length = len(df_jan)
total_jan_length

3066766

In [10]:
df_jan = df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)]

In [11]:
# Length after filtering outliers
filtered_jan_length = len(df_jan)
filtered_jan_length

3009173

In [12]:
# Fraction
filtered_jan_length / total_jan_length

0.9812202822125979

### Answer:
Fraction of records left after dropping outliers: %98

***

## Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
* Fit a dictionary vectorizer
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [13]:
from sklearn.feature_extraction import DictVectorizer

In [14]:
categorical = ["PULocationID", "DOLocationID"]

In [15]:
df_jan.loc[:, categorical] = df_jan[categorical].astype(str)

<ipython-input-15-ab25b05d5da0>:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['161' '43' '48' ... '114' '230' '262']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_jan.loc[:, categorical] = df_jan[categorical].astype(str)
<ipython-input-15-ab25b05d5da0>:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['141' '237' '238' ... '239' '79' '143']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_jan.loc[:, categorical] = df_jan[categorical].astype(str)


In [16]:
jan_dicts = df_jan.loc[:, categorical].to_dict(orient="records")

In [17]:
dv = DictVectorizer(sparse=True)

In [18]:
jan_matrix = dv.fit_transform(jan_dicts)
jan_matrix.shape

(3009173, 515)

### Answer:
Number of columns is 515

## Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters, where duration is the response variable.

Calculate the RMSE of the model on the training data.

What's the RMSE on train?

In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [20]:
lr = LinearRegression()
jan_target = df_jan.duration.values

In [21]:
lr.fit(jan_matrix, jan_target)

LinearRegression()

In [22]:

y_pred_jan = lr.predict(jan_matrix)
jan_rmse = root_mean_squared_error(jan_target, y_pred_jan)
jan_rmse

7.649261932106969

### Answer:
RMSE is 7.64

## Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [23]:
df_feb = pd.read_parquet("./data/yellow_tripdata_2023-02.parquet")

In [24]:
df_feb['duration'] = df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime
df_feb.duration = df_feb.duration.apply(lambda td: td.total_seconds() / 60.0)

In [25]:
df_feb = df_feb[(df_feb.duration >= 1) & (df_feb.duration <= 60)]

In [26]:
categorical = ["PULocationID", "DOLocationID"]

In [27]:
df_feb.loc[:, categorical] = df_feb[categorical].astype(str)

<ipython-input-27-75e4354d2ae3>:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['142' '132' '161' ... '158' '79' '161']' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df_feb.loc[:, categorical] = df_feb[categorical].astype(str)
<ipython-input-27-75e4354d2ae3>:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['163' '26' '145' ... '143' '162' '140']' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df_feb.loc[:, categorical] = df_feb[categorical].astype(str)


In [28]:
feb_dicts = df_feb.loc[:, categorical].to_dict(orient="records")

In [29]:
feb_matrix = dv.transform(feb_dicts)
feb_matrix.shape

(2855951, 515)

In [30]:
feb_target = df_feb.duration.values

In [31]:
y_pred_feb = lr.predict(feb_matrix)

In [32]:
feb_rmse = root_mean_squared_error(feb_target, y_pred_feb)
feb_rmse

7.811818743246608

### Answer:
RMSE of the validation set is 7.81